In [1]:
from dotenv import load_dotenv
load_dotenv()

False

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

In [3]:
import lightning as L

from lightning.pytorch.callbacks import ModelCheckpoint

from src.model.modeling_char_encoder import LitCharEncoder
from src.data.dataset import get_train_dataloader, get_dev_dataloader, get_test_dataloader

In [4]:
SEED=42
DATASET_NAME = 'jwengr/M-ABSA-ko-noise'
SPACE_TOKEN = '[SEP]'
UNK_TOKEN = '[UNK]'
PAD_TOKEN = '[PAD]'
MINI_BATCH_SIZE=4
N_BATCH = 4
BASE_MODEL_NAME='google-bert/bert-base-multilingual-cased'
EPOCHS=10
LEARNING_RATE = 5e-5
TRAIN_MAX_LENGTH=128
VALID_MAX_LENGTH=128
INFERENCE_SENTENCE_MAX_LENGTH=64
INFERENCE_SENTENCE_MIN_LENGTH=32
INFERENCE_SENTENCE_N_OVERLAP=3

In [5]:
L.seed_everything(SEED)

Seed set to 42


42

In [6]:
train_dl = get_train_dataloader(DATASET_NAME, batch_size=MINI_BATCH_SIZE, max_length=TRAIN_MAX_LENGTH)
dev_dl = get_dev_dataloader(DATASET_NAME, batch_size=MINI_BATCH_SIZE, max_length=VALID_MAX_LENGTH)
test_dl = get_test_dataloader(DATASET_NAME, batch_size=1)

README.md:   0%|          | 0.00/595 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.64M [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/423k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/697k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14155 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/3510 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6080 [00:00<?, ? examples/s]

In [7]:
lit_char_encoder = LitCharEncoder(
    base_model_name=BASE_MODEL_NAME,
    space_token=SPACE_TOKEN,
    unk_token=UNK_TOKEN,
    pad_token=PAD_TOKEN,
    lr=LEARNING_RATE,
    epochs=EPOCHS,
    inference_sentence_max_length=INFERENCE_SENTENCE_MAX_LENGTH,
    inference_sentence_min_length=INFERENCE_SENTENCE_MIN_LENGTH,
    inference_sentence_n_overlap=INFERENCE_SENTENCE_N_OVERLAP
)

/home/jjw1214/.local/share/virtualenvs/KROP-sezSv-b5/lib/python3.12/site-packages/transformers/models/auto/modeling_auto.py:2178: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you e

In [8]:
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/charencoder',
    filename=f"{DATASET_NAME.split('/')[1]}/{BASE_MODEL_NAME.split('/')[1]}"+"-{epoch:02d}-{valid_loss:.4f}",
    every_n_epochs=1,
    save_top_k=-1,
)

In [9]:
trainer = L.Trainer(
    callbacks=[checkpoint_callback],
    precision='16-mixed',
    max_epochs=EPOCHS,
    enable_checkpointing=True,
    accumulate_grad_batches=N_BATCH
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(lit_char_encoder, train_dl, dev_dl)

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/jjw1214/.local/share/virtualenvs/KROP-sezSv-b5/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:658: Checkpoint directory /home/jjw1214/github/KROP/checkpoints/charencoder exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]

  | Name    | Type        | Params | Mode 
------------------------------------------------
0 | encoder | CharEncoder | 177 M  | train
------------------------------------------------
177 M     Trainable params
0         Non-trainable params
177 M     Total params
711.898   Total estimated model params size (MB)
1         Modules in train mode
233       Module

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/jjw1214/.local/share/virtualenvs/KROP-sezSv-b5/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.
/home/jjw1214/.local/share/virtualenvs/KROP-sezSv-b5/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [6]:
lit_char_encoder = LitCharEncoder.load_from_checkpoint(
    'checkpoints/charencoder/DACON-Korean-Review-Obfuscation/bert-base-multilingual-cased-epoch=02-valid_loss=0.6253.ckpt',
    base_model_name=BASE_MODEL_NAME,
    space_token=SPACE_TOKEN,
    unk_token=UNK_TOKEN,
    pad_token=PAD_TOKEN,
    lr=LEARNING_RATE,
    epochs=EPOCHS,
    inference_sentence_max_length=INFERENCE_SENTENCE_MAX_LENGTH,
    inference_sentence_min_length=INFERENCE_SENTENCE_MIN_LENGTH,
    inference_sentence_n_overlap=INFERENCE_SENTENCE_N_OVERLAP
)

c:\Users\jinwo\.virtualenvs\KROP-L3im0CPD\Lib\site-packages\transformers\models\auto\modeling_auto.py:2178: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly id

In [7]:
trainer = L.Trainer()

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [8]:
preds = trainer.predict(lit_char_encoder, test_dl)

You are using a CUDA device ('NVIDIA GeForce RTX 5070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\jinwo\.virtualenvs\KROP-L3im0CPD\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

온션퓨 갸셩피 쑥쏘웩오! 별섶 듀 펀쳅 빵물닒예옷! 힐륑햐려 왔눈뎨 완젼 힐륑하꾜 깝뉘댜ㅎㅎ 부됴 념뮤 옜푸꼲 
오션뷰[사EP]ㆀ성비[사EP]숙소네요![사EP]버코[사EP]두[사EP]펀ㆀ[사EP]방모빵에요! 힐링하려[사EP]왔는데[사EP]와강[사EP]힐링하ㆀ[사EP]갑니다니ㆀ[사EP]부가[사EP]너무[사EP]바푸탑[사EP]


IndexError: string index out of range

In [ ]:
for batch in test_dl:
    break

In [ ]:
batch

{'sentence_noisy': ['온션퓨 갸셩피 쑥쏘웩오! 별섶 듀 펀쳅 빵물닒예옷! 힐륑햐려 왔눈뎨 완젼 힐륑하꾜 깝뉘댜ㅎㅎ 부됴 념뮤 옜푸꼲 싣셜도 칼큼햐교 쥑건뿐둘또 췬졀햐쎄오'],
 'sentence': ['오션뷰 가성비 숙소에요! 벌써 두 번째 방문이에요! 힐링하러 왔는데 완전 힐링하고 갑니다ㅎㅎ 뷰도 너무 예쁘고 시설도 깔끔하고 직원분들도 친절하세요']}

In [ ]:
preds[0]

['오션뷰 가성비 숙소예요! 별게 두 번째 방문이에요! 힐링하러 왔는데 완전 힐링하고 갑니다[UNK] 뷰가 너무 예쁘에 시설도 깔끔하고 직원분들도 친절하세요']